In [79]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

# Steam Regression Analysis

## Steam Games Dataset

The Steam dataset originates from https://data.world/craigkelly/steam-game-data and was created by Craig Kelly. The dataset includes ownership data on around 12,000 videogames on the Steam digital distribution platform, and features such as genre, full text descriptions (written by the publisher/developer of the game), descriptive tags, supported languages, release dates, and reviewer scores. I'm interested in this dataset because I intend to publish a game within the next few years and I'm curious as to whether or not certain features are correlated with game popularity.

In [3]:
steam = pd.read_csv("data/games-features.csv")

In [4]:
print(steam.columns)

Index(['QueryID', 'ResponseID', 'QueryName', 'ResponseName', 'ReleaseDate',
       'RequiredAge', 'DemoCount', 'DeveloperCount', 'DLCCount', 'Metacritic',
       'MovieCount', 'PackageCount', 'RecommendationCount', 'PublisherCount',
       'ScreenshotCount', 'SteamSpyOwners', 'SteamSpyOwnersVariance',
       'SteamSpyPlayersEstimate', 'SteamSpyPlayersVariance',
       'AchievementCount', 'AchievementHighlightedCount', 'ControllerSupport',
       'IsFree', 'FreeVerAvail', 'PurchaseAvail', 'SubscriptionAvail',
       'PlatformWindows', 'PlatformLinux', 'PlatformMac', 'PCReqsHaveMin',
       'PCReqsHaveRec', 'LinuxReqsHaveMin', 'LinuxReqsHaveRec',
       'MacReqsHaveMin', 'MacReqsHaveRec', 'CategorySinglePlayer',
       'CategoryMultiplayer', 'CategoryCoop', 'CategoryMMO',
       'CategoryInAppPurchase', 'CategoryIncludeSrcSDK',
       'CategoryIncludeLevelEditor', 'CategoryVRSupport', 'GenreIsNonGame',
       'GenreIsIndie', 'GenreIsAction', 'GenreIsAdventure', 'GenreIsCasual',
       'G

In [70]:
steam.shape

(13357, 78)

In [5]:
steam.head()

,QueryID,ResponseID,QueryName,ResponseName,ReleaseDate,RequiredAge,DemoCount,DeveloperCount,DLCCount,Metacritic,...,LegalNotice,Reviews,SupportedLanguages,Website,PCMinReqsText,PCRecReqsText,LinuxMinReqsText,LinuxRecReqsText,MacMinReqsText,MacRecReqsText
0,10,10,Counter-Strike,Counter-Strike,Nov 1 2000,0,0,1,0,88,...,,,English French German Italian Spanish Simplifi...,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
1,20,20,Team Fortress Classic,Team Fortress Classic,Apr 1 1999,0,0,1,0,0,...,,,English French German Italian Spanish,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
2,30,30,Day of Defeat,Day of Defeat,May 1 2003,0,0,1,0,79,...,,,English French German Italian Spanish,http://www.dayofdefeat.com/,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
3,40,40,Deathmatch Classic,Deathmatch Classic,Jun 1 2001,0,0,1,0,0,...,,,English French German Italian Spanish,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
4,50,50,Half-Life: Opposing Force,Half-Life: Opposing Force,Nov 1 1999,0,0,1,0,0,...,,,English French German Korean,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,


In [6]:
print(steam.iloc[100])

QueryID                                                          3170
ResponseID                                                       3170
QueryName                             King's Bounty: Armored Princess
ResponseName                           Kings Bounty: Armored Princess
ReleaseDate                                               Nov 19 2009
                                          ...                        
PCRecReqsText       Recommended:OS: Windows XP/Vista Processor: 3 ...
LinuxMinReqsText                                                     
LinuxRecReqsText                                                     
MacMinReqsText      OS: Mac OSX 10.6 or higher                Proc...
MacRecReqsText                                                       
Name: 100, Length: 78, dtype: object


In [7]:
steam['PriceCurrency'].value_counts()

USD    10739
        2618
Name: PriceCurrency, dtype: int64

In [8]:
steam['ResponseName'].value_counts().head(30)

Train Simulator 2016                              9
Jewel Quest Pack                                  7
Jagged Alliance 2 Gold                            5
SiN Episodes: Emergence                           5
F.E.A.R.                                          5
Mahjong Quest Collection                          5
Cake Mania Collection                             5
Tom Clancys Rainbow Six(r) Siege                  5
Crazy Machines 1.5                                5
BRINK                                             5
RIP - Trilogy(tm)                                 5
Westward Collection                               5
Super Granny Collection                           5
Tradewinds Classics                               5
Crysis Warhead(r)                                 3
Shank                                             3
Portal 2                                          3
The Sims(tm) 3                                    3
Civilization IV: Beyond the Sword                 3
Trine 2: Com

In [9]:
#steam['ExtUserAcctNotice'].value_counts()
#steam['DRMNotice'].value_counts()
#steam['Background'].value_counts()
#steam['HeaderImage'].value_counts()
#steam['Reviews'].value_counts()
#steam['PCMinReqsText'].value_counts()
steam['ReleaseDate'].value_counts()

Aug 22 2016      241
Aug 26 2016      152
                  87
Sep 9 2016        77
Sep 7 2016        68
                ... 
Mar 19 2012        1
Oct 1 2012         1
Sep 25 2008        1
Early October      1
Mar 27 2007        1
Name: ReleaseDate, Length: 2117, dtype: int64

### Notes on Steam dataset

Category is already encoded.

It looks like all prices are in USD, though some are unlabeled I'm just going to assume that they are USD and the webscraper didn't retrieve them correctly.

For now textual descriptions such as Background, AboutText, SupportedLanguages, and MinReqsText will all be removed as I do not yet know how to perform a bag-of-words or word-vector style feature decomposition on these. I will probably add them back in later when I've done more research on ohw to do this.

Links like Background or HeaderImage will also be removed.

Note that these removed columns probably do actually contain very valuable data to making an accurate prediction, it's just that I currently do not know how to incorporate them into regression and/or it would be infeasible to donwload the content of all of these links for 40,000 examples in a short amount of time. I might come back to that though??

There appear to be duplicate games??? I'll have to think of a creative way to merge these.

In [107]:
def steam_preprocess(df):
    ret = df.drop(labels=[
        'QueryID',
        'ResponseID',
        'QueryName',
        'ResponseName', # Could be useful for 'catchiness' or 'length', but that's a whole can of worms
        'SupportEmail',
        'SupportURL',
        'Background', # boolean
        'ShortDescrip',
        'DetailedDescrip',
        'AboutText',
        'PriceCurrency',
        'DRMNotice', # useful to convert to boolean value
        'ExtUserAcctNotice', # useful to convert to boolean value
        'HeaderImage', # boolean
        'LegalNotice', # boolean
        'Reviews', # boolean
        'SupportedLanguages', # Could separate up into a series of encoded variables, remove for now
        'Website', # boolean
        'PCMinReqsText', # All of these could have sentiment analysis done on them
        'PCRecReqsText',
        'LinuxMinReqsText',
        'LinuxRecReqsText',
        'MacMinReqsText',
        'MacRecReqsText',
        'SteamSpyPlayersEstimate', # The next three are dead giveaways and would give the model too much information.
        'SteamSpyPlayersVariance',
        'SteamSpyOwnersVariance', # Maybe could be incorporated into evaluation in some way
    ], axis=1)
    
    ret['ReleaseDate'] = pd.to_datetime(ret['ReleaseDate'], errors='coerce')
    
    # I would rather remove games that have no release date rather than impute them. Knowing steam, they probably aren't released yet, and imputation makes no sense here.
    # I might replace it with iterative imputation, but as it stands I'm losing like 400 games out of the 40,000 samples. I'd rather just not train on them.
    ret.dropna(subset=['ReleaseDate'], inplace=True)
    
    ret['ReleaseYear'] = pd.DatetimeIndex(ret['ReleaseDate']).year
    ret['ReleaseMonth'] = pd.DatetimeIndex(ret['ReleaseDate']).month
    ret['ReleaseDay'] = pd.DatetimeIndex(ret['ReleaseDate']).day
    ret = ret.drop(['ReleaseDate'], axis=1)
    return ret

steam_cleaned = steam_preprocess(steam)
steam_cleaned.head() # MINDBLOWINGLY, SHOCKINGLY CLEAN

,RequiredAge,DemoCount,DeveloperCount,DLCCount,Metacritic,MovieCount,PackageCount,RecommendationCount,PublisherCount,ScreenshotCount,...,GenreIsEarlyAccess,GenreIsFreeToPlay,GenreIsSports,GenreIsRacing,GenreIsMassivelyMultiplayer,PriceInitial,PriceFinal,ReleaseYear,ReleaseMonth,ReleaseDay
0,0,0,1,0,88,0,1,68991,1,13,...,False,False,False,False,False,9.99,9.99,2000,11,1
1,0,0,1,0,0,0,1,2439,1,5,...,False,False,False,False,False,4.99,4.99,1999,4,1
2,0,0,1,0,79,0,1,2319,1,5,...,False,False,False,False,False,4.99,4.99,2003,5,1
3,0,0,1,0,0,0,1,888,1,4,...,False,False,False,False,False,4.99,4.99,2001,6,1
4,0,0,1,0,0,0,1,2934,1,5,...,False,False,False,False,False,4.99,4.99,1999,11,1


In [108]:
steam_cleaned.shape

(12872, 53)

In [109]:
X = steam_cleaned.drop(['SteamSpyOwners'], axis=1)
y = steam_cleaned['SteamSpyOwners']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

## Linear Regression Implementation and Comparison

Compares LinReg, a hand-implemented LinReg implementation, and SGDReg.

The former two use the normal equation for linear regression, while the latter uses gradient descent. Comparison of times and coefficients is provided.

In [69]:
from sklearn.linear_model import SGDRegressor, LinearRegression

# No hyperparameter tuning

start = time.time()
reg = SGDRegressor(alpha=0, max_iter=10000).fit(X_train, y_train)
print("Time elapsed: ", time.time() - start)

reg.coef_

Time elapsed:  0.1640174388885498


array([ 1.39098810e+12,  2.93885866e+10,  8.43524008e+10,  5.45791656e+12,
        1.32013793e+12,  2.19745800e+11,  5.99540643e+10, -5.99758202e+11,
        4.89763859e+10, -1.52912601e+11,  2.02310136e+12,  1.72715351e+12,
        1.04488489e+11,  9.96744925e+10, -1.84808846e+09,  4.07300659e+10,
       -1.17322555e+10,  8.54628500e+08, -1.58096244e+10,  4.59416817e+09,
        5.96478292e+10,  1.22801297e+11,  8.01905673e+10,  1.19316514e+11,
        5.05680618e+10,  2.86896612e+10,  2.30924111e+10,  2.10305014e+10,
        6.20797487e+10,  1.91830786e+10,  3.69097927e+08, -1.35612148e+10,
        2.07905006e+10,  3.54039558e+09,  3.79946007e+10, -9.88167816e+09,
        1.74466802e+11,  6.56921233e+10, -1.60553329e+11, -6.74190875e+09,
        5.44405799e+10,  5.10184839e+10, -5.69369121e+10,  9.87110282e+10,
       -1.53964110e+09,  5.98451787e+10,  4.28137740e+10,  1.15398286e+12,
       -3.23726590e+11,  4.40867407e+11, -1.48969702e+11, -2.22542199e+10])

In [64]:
reg = LinearRegression().fit(X_train, y_train)

reg.coef_

array([ 4.20256403e+03, -7.63442301e+04,  2.90665190e+04, -2.27728811e+02,
        2.50194944e+03,  4.26126994e+04, -1.70327480e+04,  4.38456307e+01,
        8.18796316e+04, -9.38328409e+02,  5.65521655e+02, -1.06361745e+03,
       -3.09671805e+04,  1.59895424e+05, -3.19749632e+05,  1.77358819e+04,
       -1.92346838e+03, -6.58036770e+04,  1.62201595e+05,  6.08913778e+04,
       -8.21429279e+04,  4.32664565e+04, -5.57853252e+04, -7.83650053e+04,
        1.68617751e+04, -4.06928884e+04, -2.21971576e+05, -4.81333461e+04,
        1.57414436e+05,  7.10343929e+04,  4.04852633e+05,  2.48698854e+06,
        4.37185789e+04,  7.36768985e+04, -1.43228875e+05, -5.24268691e+04,
        1.00636289e+05,  1.82773742e+03,  6.37832475e+03, -5.42835224e+02,
        5.04873491e+03,  3.04002883e+04,  2.47132874e+04,  3.76975868e+05,
       -6.42186124e+03, -3.44353182e+04, -2.68425127e+05,  2.32245804e+02,
       -1.41604594e+03, -7.20753926e+04, -2.64897022e+03, -2.35702064e+03])

In [56]:
from sklearn.utils.estimator_checks import check_estimator
from sklearn.base import BaseEstimator, RegressorMixin
# Custom linear regression

class GhettoLinearRegression(BaseEstimator, RegressorMixin):
    coef_ = None
    
    def _pad_column(self, X): # Adds column of 1's at the beginning for the intercept
        X = X.to_numpy(dtype='float')
        return np.insert(X, 0, 1, axis=1)
        #return np.append(X, 1, axis=1)
    
    def fit(self, X, y):
        X = self._pad_column(X)
        self.coef_ = np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))
        return self
    
    def predict(self, X):
        X = self._pad_column(X)
        return X.dot(self._coef)
    
    def score(self, X, y):
        pred = self.predict(X)
        return (pred - y)**2

In [63]:
start = time.time()
reg = GhettoLinearRegression().fit(X_train, y_train)
print("Time elapsed: ", time.time() - start)

reg.coef_

Time elapsed:  0.08312153816223145


array([ 1.45427091e+08,  4.20256403e+03, -7.63442301e+04,  2.90665190e+04,
       -2.27728811e+02,  2.50194944e+03,  4.26126994e+04, -1.70327480e+04,
        4.38456307e+01,  8.18796316e+04, -9.38328409e+02,  5.65521655e+02,
       -1.06361745e+03, -3.09671805e+04,  1.59895424e+05, -3.19749632e+05,
        1.77358819e+04, -1.92346839e+03, -6.58036767e+04,  1.62201595e+05,
        6.08913778e+04, -8.21429279e+04,  4.32664565e+04, -5.57853252e+04,
       -7.83650053e+04,  1.68617750e+04, -4.06928884e+04, -2.21971576e+05,
       -4.81333461e+04,  1.57414436e+05,  7.10343929e+04,  4.04852633e+05,
        2.48698854e+06,  4.37185789e+04,  7.36768985e+04, -1.43228875e+05,
       -5.24268691e+04,  1.00636289e+05,  1.82773742e+03,  6.37832475e+03,
       -5.42835225e+02,  5.04873491e+03,  3.04002883e+04,  2.47132874e+04,
        3.76975868e+05, -6.42186124e+03, -3.44353182e+04, -2.68425127e+05,
        2.32245804e+02, -1.41604594e+03, -7.20753926e+04, -2.64897022e+03,
       -2.35702064e+03])

## Logistic Regression Regularization Comparison

In [94]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

scores = cross_val_score(clf, X_train, y_train)

/home/dylan/.build/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/dylan/.build/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/dylan/.build/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the nu

KeyboardInterrupt: 

In [ ]:
scores

In [85]:

search = GridSearchCV(LogisticRegression(), [
    {'penalty': ['l2', 'l1'], 'C': [0.001, 0.1, 1]},
    {'penalty': ['elasticnet'], 'C': [0.001, 0.1, 1], 'l1_ratio': [0.05, 0.5, 0.95]}
],scoring='neg_mean_squared_error', cv=KFold(2), n_jobs=4, verbose=10)

search = search.fit(X_train, y_train)

Fitting 2 folds for each of 15 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done  27 out of  30 | elapsed:  1.5min remaining:    9.9s
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:  2.1min finished
/home/dylan/.build/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [86]:
search.best_params_

{'C': 1, 'penalty': 'l2'}

In [92]:
search.cv_results_['mean_test_score']

array([-7.23010264e+12,             nan, -7.49176937e+12,             nan,
       -5.97248194e+12,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan])

## KNN Implementation

In [252]:
from sklearn.preprocessing import StandardScaler

class GhettoKNN(BaseEstimator, RegressorMixin):
    def __init__(self, k=1, p=2):
        self.k_ = k
        self.p_ = p
        #self.scaler_ = StandardScaler()
    X_ = None
    y_ = None
    def dist(self, x1, x2):
        #print('x1:',  x1)
        #print('x2:', x2)
        ab = np.absolute(x2 - x1)
        #print('ab:', ab)
        pw = ab**self.p_
        #print('pw:', pw)
        sm = np.sum(pw, axis=1)
        #print('sm:', sm)
        fn = sm ** (1.0 / self.p_)
        #print('fn:', fn)
        return fn
    
    def fit(self, X, y):
        #self.X_ = self.scaler_.fit_transform(X.to_numpy(dtype='float'))
        self.X_ = X.to_numpy(dtype='float')
        #print(self.X_.shape)
        #print(len(self.X_))
        self.y_ = y.to_numpy(dtype='float')
        return self
        
    def predict(self, X):
        X = X.to_numpy(dtype='float')
        #print(np.sum(X))
        # Go one example at a time
        predictions = []
        for example in X:
            #print(example)
            # Make an array of repeating examples
            t = np.tile(example, (np.size(self.X_, 0), 1))
            #print(t.shape)
            #print(example[0])
            # Calculate distance between example and all stored training examples
            distances = self.dist(t, self.X_)
            #print(distances[:100])
            # Get indices of k smallest distances
            idx = np.argpartition(distances, self.k_)
            #print(idx.shape)
            predictions.append(sum(self.y_[idx[:self.k_]]) / self.k_)
        return predictions
            

In [253]:
reg = GhettoKNN()
reg.fit(X_train, y_train)
print(X_train.shape)

(10297, 52)


In [254]:
predictions = reg.predict(X_train)

In [255]:
predictions[:100]

[1077.0,
 0.0,
 72816.0,
 0.0,
 196567.0,
 132360.0,
 6457.0,
 22239.0,
 25288.0,
 0.0,
 14886.0,
 1605.0,
 36049.0,
 3587.0,
 13810.0,
 34076.0,
 814606.0,
 0.0,
 206252.0,
 0.0,
 36946.0,
 535.0,
 43761.0,
 78555.0,
 9506.0,
 0.0,
 312786.0,
 40533.0,
 1174560.0,
 10223.0,
 78376.0,
 0.0,
 652653.0,
 4663.0,
 0.0,
 0.0,
 1435.0,
 45734.0,
 0.0,
 1614.0,
 0.0,
 5022.0,
 4484.0,
 85550.0,
 12016.0,
 1076.0,
 97746.0,
 3766.0,
 7891.0,
 0.0,
 4125.0,
 1973.0,
 33359.0,
 14348.0,
 2152.0,
 57751.0,
 19011.0,
 5919.0,
 25647.0,
 0.0,
 10582.0,
 0.0,
 44299.0,
 339509.0,
 10044.0,
 2690.0,
 0.0,
 1793.0,
 1435.0,
 0.0,
 0.0,
 803.0,
 0.0,
 97207.0,
 0.0,
 0.0,
 0.0,
 26544.0,
 138458.0,
 1973.0,
 0.0,
 4842.0,
 58827.0,
 180605.0,
 140969.0,
 22060.0,
 63490.0,
 539.0,
 648169.0,
 16321.0,
 6636.0,
 35153.0,
 24930.0,
 0.0,
 5201.0,
 11299.0,
 235665.0,
 3946.0,
 3408.0,
 312427.0]

In [256]:
y_train.to_numpy()[:100]

array([   1077,       0,   72816,       0,  196567,  132360,    6457,
         22239,   25288,       0,   14886,    1605,   36049,    3587,
         13810,   34076,  814606,       0,  206252,       0,   36946,
           535,   43761,   78555,    9506,       0,  312786,   40533,
       1174560,   10223,   78376,       0,  652653,    4663,       0,
             0,    1435,   45734,       0,    1614,       0,    5022,
          4484,   85550,   12016,    1076,   97746,    3766,    7891,
             0,    4125,    1973,   33359,   14348,    2152,   57751,
         19011,    5919,   25647,       0,   10582,       0,   44299,
        339509,   10044,    2690,       0,    1793,    1435,       0,
             0,     803,       0,   97207,       0,       0,       0,
         26544,  138458,    1973,       0,    4842,   58827,  180605,
        140969,   22060,   63490,     539,  648169,   16321,    6636,
         35153,   24930,       0,    5201,   11299,  235665,    3946,
          3408,  312

In [257]:
from sklearn.neighbors import KNeighborsRegressor

reg = KNeighborsRegressor(n_neighbors=2).fit(X, y)

reg.predict(X_train)[:100]

array([1.973500e+03, 8.025000e+02, 1.123625e+05, 1.435000e+03,
       3.409435e+05, 1.110175e+05, 9.326500e+03, 2.753000e+04,
       6.465550e+04, 0.000000e+00, 3.416600e+04, 1.337500e+03,
       1.856250e+04, 1.246500e+04, 6.905000e+03, 2.107350e+04,
       4.126835e+05, 9.430000e+02, 2.423015e+05, 0.000000e+00,
       1.901100e+04, 3.585500e+03, 2.761950e+04, 4.268500e+04,
       7.174000e+03, 2.511000e+03, 1.954910e+05, 5.362550e+04,
       1.174560e+06, 2.376400e+04, 7.703100e+04, 0.000000e+00,
       4.486430e+05, 3.497500e+03, 0.000000e+00, 0.000000e+00,
       1.076100e+04, 8.967450e+04, 2.152000e+03, 1.076000e+03,
       0.000000e+00, 2.448150e+04, 1.174750e+04, 7.658250e+04,
       6.725500e+03, 9.395000e+02, 4.923150e+04, 3.676500e+03,
       1.363050e+04, 3.362800e+04, 5.022000e+03, 3.407500e+03,
       2.143250e+04, 9.864000e+03, 3.587000e+03, 3.093800e+04,
       2.403300e+04, 2.959500e+03, 1.416850e+04, 2.690000e+02,
       8.788500e+03, 0.000000e+00, 2.905450e+04, 6.4906

In [258]:
reg = GhettoKNN(k=2).fit(X, y)

reg.predict(X_train)[:100]

[1973.5,
 802.5,
 112362.5,
 1435.0,
 340943.5,
 111017.5,
 9326.5,
 27530.0,
 64655.5,
 0.0,
 34166.0,
 1337.5,
 18562.5,
 12465.0,
 6905.0,
 21073.5,
 412683.5,
 943.0,
 242301.5,
 0.0,
 19011.0,
 3585.5,
 27619.5,
 42685.0,
 7174.0,
 2511.0,
 195491.0,
 53625.5,
 1174560.0,
 23764.0,
 77031.0,
 0.0,
 448643.0,
 3497.5,
 0.0,
 0.0,
 10761.0,
 89674.5,
 2152.0,
 1076.0,
 0.0,
 24481.5,
 11747.5,
 76582.5,
 6725.5,
 939.5,
 49231.5,
 3676.5,
 13630.5,
 33628.0,
 5022.0,
 3407.5,
 21791.0,
 9864.0,
 3587.0,
 30938.0,
 24033.0,
 2959.5,
 14168.5,
 269.0,
 8788.5,
 0.0,
 29054.5,
 646914.0,
 18294.0,
 9595.0,
 0.0,
 1972.5,
 717.5,
 0.0,
 0.0,
 401.5,
 4394.0,
 58198.5,
 1166.0,
 0.0,
 0.0,
 26275.0,
 89047.0,
 13810.0,
 0.0,
 5470.0,
 47797.0,
 303818.0,
 84743.0,
 47707.0,
 46182.5,
 269.5,
 573739.0,
 21791.0,
 4573.5,
 23315.5,
 44210.0,
 0.0,
 9595.0,
 32910.5,
 120612.5,
 7802.0,
 1704.0,
 468909.5]

In [259]:
X_train_scaled = StandardScaler().fit_transform(X_train)

In [260]:
for reg in [GhettoKNN(k=5), KNeighborsRegressor(n_neighbors=5)]:
    reg.fit(X_train_scaled, y_train)
    predictions = reg.predict(X_train)
    print(predictions[:20])

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'